In [60]:
# Import packages
import pandas as pd
import sqlite3


# SQL Query
conn = None
try:
    conn = sqlite3.connect("SVY05 - Design Col.db") # TODO create prompt for this
except:
    print("error")

In [61]:
#brace frame columns
#ignored for gravity analysis
bf_cols = ["A-2","A-3","A-4","A-5","A-6","A-7",
                "H-1","H-2","H-2.7","H-5","H-6","H-7",
                "K-2","K-3",
                "F.2-1","E.4-1","D.5-1","C.5-1",
                "F.2-3","E.4-3","D.5-3","C.5-3",
                "F-8","E-8","D-8","C-8",
                "K-5.5","J-5.5",
                "H.8-1","H-1"]
bf_cols_to_sql = format(str(bf_cols)[1:-1])

In [120]:
import timeit
#iterate over floor levels
start = time.time()
cursor = conn.cursor()

cursor.execute("""SELECT Story_UID, Level, Story_Label, FloorHeight
        FROM StoryData
        ORDER BY Level
        """)

qq_gl = f"""
        SELECT *
        FROM (SELECT DISTINCT REPLACE(c1.Grid_Label, ' ','') AS GL, c1.Grid_Label
        FROM ColumnData AS c1) AS gg
        WHERE gg.GL NOT IN ({bf_cols_to_sql})
                """


ddff = pd.read_sql(qq_gl, conn)
floor_dict = {}
for row in cursor:
        floor_dict[row[2]] = {
                                'Level':row[1],
                                'Story_Label':row[2],
                                'FloorHeight':row[3],
                                'Story_UID':row[0]
                              }
         
        qq =f"""
                SELECT
                        gg.GL,
                        \"{row[2]}\"
                FROM (
                        SELECT 
                                REPLACE(c1.Grid_Label, ' ','') AS GL,
                                c1.Grid_Label,
                                c1.Size \"{row[2]}\"
                        FROM ColumnData AS c1
                        WHERE c1.Story_UID = {row[0]}    
                        ) AS gg
                WHERE gg.GL NOT IN ({bf_cols_to_sql})"""
        df1 = pd.read_sql(qq, conn)
        ddff = ddff.merge(df1, on=['GL'], how='left')
end = time.time()
print(end - start)

0.01196908950805664


Check with performance of SQL only solution with hardkeyed column names

In [121]:
#Check performance of SQL only solution
start = time.time()
query_column = f"""
        SELECT Story_UID, Level, Story_Label, FloorHeight
        FROM StoryData
        ORDER BY Level
                """
query_column = f"""

                WITH st AS(
                        SELECT Story_UID, Level, Story_Label, FloorHeight
                        FROM StoryData
                        ORDER BY Level
                )
                
                
                SELECT
                        gg.GL,
                        sz_hr, sz_4th, sz_3rd, sz_2nd
                FROM (
                        SELECT 
                                REPLACE(c1.Grid_Label, ' ','') AS GL, c1.Grid_Label, c2.Grid_Label, c3.Grid_Label,
                                c1.Size AS sz_2nd,
                                c2.Size AS sz_3rd,
                                c3.Size AS sz_4th,
                                c4.Size AS sz_hr,
                                g.AxialDL, g.AxialSW, g.AxialPosLLRed, g.PosLLRF, g.AxialPosLLNonRed
                        FROM ColumnData AS c1
                        LEFT JOIN GravLoadReactionsOnColumn AS g ON g.Column_UID = c1.Column_UID
                        LEFT JOIN ColumnData AS c2 ON c1.Grid_Label = c2.Grid_Label AND c2.Story_UID = 4027
                        LEFT JOIN ColumnData AS c3 ON c1.Grid_Label = c3.Grid_Label AND c3.Story_UID = 2027
                        LEFT JOIN ColumnData AS c4 ON c1.Grid_Label = c4.Grid_Label AND c4.Story_UID = 1027
                        WHERE c1.Story_UID = 3027          
                        ) AS gg
                WHERE gg.GL NOT IN ({bf_cols_to_sql})
                ORDER BY gg.GL DESC
                """

dff = pd.read_sql(query_column, conn)
end = time.time()
print(end - start)

0.0039882659912109375


The SQL-only query is over 10x more performant
The repeated Pandas merge call to left join is likely a large culprit
We should minimize the amount of data transactions between SQLite3 and Pandas in anticipation of larger structural models

To do this, we will convert the list of tuples for each run of the floor-level loop
into a dictionary that can be read as a dataframe just one time, at the end of the loop

In [125]:
#iterate over floor levels
start = time.time()
cursor = conn.cursor()

floor_dict = {}

qq_gl = f"""
        SELECT *
        FROM (SELECT DISTINCT REPLACE(c1.Grid_Label, ' ','') AS GL, c1.Grid_Label
        FROM ColumnData AS c1) AS gg
        WHERE gg.GL NOT IN ({bf_cols_to_sql})
                """
cursor.execute("""SELECT Story_UID, Level, Story_Label, FloorHeight
        FROM StoryData
        ORDER BY Level
        """)

level_to_gl={}
for row in cursor:
        floor_dict[row[2]] = {
                                'Level':row[1],
                                'Story_Label':row[2],
                                'FloorHeight':row[3],
                                'Story_UID':row[0]
                              }
        
        cur = conn.cursor()
        qq =f"""
                SELECT
                        gg.GL,
                        \"{row[2]}\"
                FROM (
                        SELECT 
                                REPLACE(c1.Grid_Label, ' ','') AS GL,
                                c1.Grid_Label,
                                c1.Size \"{row[2]}\"
                        FROM ColumnData AS c1
                        WHERE c1.Story_UID = {row[0]}    
                        ) AS gg
                WHERE gg.GL NOT IN ({bf_cols_to_sql})"""
        col_ls = cur.execute(qq).fetchall()
        level_to_gl[row[2]] = {i[0]:i[1] for i in col_ls}

gl_col_df = pd.DataFrame(level_to_gl)

end = time.time()
print(end - start)

0.0019943714141845703


This is even quicker than the SQL-only query and this is also more scalable because no hardkeys are made

In [126]:
gl_col_df

,2nd,3rd,Roof,High Roof
K-1,W14X90,W14X90,W14X90,W14X90
K-4.5,W14X90,W14X90,W14X90,W14X90
J-2,W14X109,W14X109,W14X90,W14X90
J-3,W14X99,W14X99,W14X90,W14X90
J-4.5,W14X90,W14X90,W14X90,W14X90
J-6.3,W14X90,W14X90,W14X90,W14X90
26.75ft-193.00ft,W14X90,W14X90,W14X90,W14X90
H.4-7,W14X90,W14X90,W14X90,W14X90
H-4,W14X193,W14X193,W14X90,W14X90
H-8,W14X90,W14X90,W14X90,NaN
